In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from datetime import datetime as dt
import numpy as np
from scipy.stats.stats import pearsonr  
from sklearn.preprocessing import normalize

plt.style.use('ggplot')
pd.options.mode.chained_assignment = None
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("data/churn_dataset.csv")
df_cluster = pd.read_csv('data/clustered_customers.csv')[['customer_db_id', 'cluster', 'cluster_name', 'freq_mean', 'freq_median']]
df = df.merge(df_cluster, on='customer_db_id', how='left')

In [9]:
df_zips = pd.read_csv("data/zips_berlin.csv", header=None)
df_zips.columns = ['city_part', 'zips']

In [11]:
df_zips['zips'] = df_zips['zips'].apply(lambda x: x.split(';'))

In [12]:
df_zips.head()

,city_part,zips
0,Mitte,"[10115, 10117, 10119, 10178, 10179, 10435, 105..."
1,Friedrichshain-Kreuzberg,"[10179, 10243, 10245, 10247, 10249, 10367, 107..."
2,Pankow,"[10119, 10247, 10249, 10405, 10407, 10409, 104..."
3,Charlottenburg-Wilmersdorf,"[10553, 10585, 10587, 10589, 10623, 10625, 106..."
4,Spandau,"[13581, 13583, 13585, 13587, 13589, 13591, 135..."


In [18]:
(df_zips.zips.apply(pd.Series)
              .stack()
              .reset_index(level=1, drop=True)
              .to_frame('zips'))

,zips
0,10115
0,10117
0,10119
0,10178
0,10179
0,10435
0,10551
0,10553
0,10555
0,10557


# Preprocessing

In [ ]:
df['churned'] = [True if x > 2 else False for x in df.churn_factor]
df['first_order_voucher_revenue_ratio'] = df.first_order_voucher_value / (df.first_order_voucher_value + df.first_order_revenue)

## Fill NAs

In [ ]:
nans = df.isnull().sum()
nans[nans > 0].sort_values(ascending = False)

In [ ]:
df['first_order_products'] = df['first_order_products'].fillna("")

In [ ]:
df[['refunds_unsuccess', 'refunds_success']] = df[['refunds_unsuccess', 'refunds_success']].fillna(0)

In [ ]:
rating_cols = ['avg_rating', 'rating_diff', 'last_order_rating', 'rated_orders', 'first_order_rating']
df[rating_cols] = df[rating_cols].fillna(-999)
df.loc[df.rated_orders > 0, rating_cols].head()

In [ ]:
df.aov = df.aov.fillna(0)
df.segment = df.segment.fillna('None')
df.gender = df.gender.fillna('Unknown')

In [ ]:
df.zip = df.zip.fillna('Unknown')
df.zip_area = df.zip_area.fillna('Unknown')

# df.avg_hub_distance = df.avg_hub_distance.fillna(-999)
# df.last_order_hub_distance = df.last_order_hub_distance.fillna(-999)
# df.first_order_hub_distance = df.first_order_hub_distance.fillna(-999)
# df.laundry_distance = df.laundry_distance.fillna(-999)
# df.laundry_rating = df.laundry_rating.fillna(-999)
# df.laundry_within_1km = df.laundry_within_1km.fillna(-999)

In [ ]:
df.first_order_date = pd.to_datetime(df.first_order_date)
df['first_order_week'] = df['first_order_date'].dt.week
df['first_order_year'] = df['first_order_date'].dt.year
df['first_order_day'] = df['first_order_date'].dt.day
df['first_order_month'] = df['first_order_date'].dt.month

In [ ]:
df.first_order_fac_name = df.first_order_fac_name.fillna('unknown')
df.first_order_voucher_channel = df.first_order_voucher_channel.fillna('unknown')
df.first_order_voucher_value = df.first_order_voucher_value.fillna(-999)
df.first_order_voucher_revenue_ratio = df.first_order_voucher_revenue_ratio.fillna(0)
df.last_order_fac_name = df.last_order_fac_name.fillna('unknown')
df.last_order_voucher_channel = df.last_order_voucher_channel.fillna('unknown')
df.last_order_voucher_value = df.last_order_voucher_value.fillna(-999)

In [ ]:
# if customer cannot be clustered because no information about itemization -> freq_median and freq_mean will be 365
df.freq_mean = df.freq_mean.fillna(365)
df.freq_median = df.freq_median.fillna(365)

In [ ]:
df = df.fillna(-999)
nans = df.isnull().sum()
nans[nans > 0].sort_values(ascending = False)

## Label Encoding

In [ ]:
voucher_channel_le = LabelEncoder()
voucher_channel_le.fit(df.first_order_voucher_channel)
df['first_order_voucher_channel_enc'] = voucher_channel_le.transform(df.first_order_voucher_channel.tolist())

products_le = LabelEncoder()
products_le.fit(df.first_order_products)
df['first_order_products_enc'] = products_le.transform(df.first_order_products.tolist())

facility_le = LabelEncoder()
facility_le.fit(df.first_order_fac_name)
df['first_order_fac_name_enc'] = facility_le.transform(df.first_order_fac_name.tolist())

order_date_le = LabelEncoder()
order_date_le.fit(df.first_order_date.sort_values().dt.strftime('%Y-%m-%d'))
df['first_order_date_enc'] = order_date_le.transform(df.first_order_date.dt.strftime('%Y-%m-%d').tolist())

# Bad vs Good Voucher Customers
*Assumption: Bad customers are customers who had one completed order using a voucher, had a good customer experience and didn't place another order for more than a year. These customers should be excluded from churn analysis.*

We want to analyse, if there are any specific features, such as area, channels, products that identify bad customers. In order to analyse, we create a comparison 'Good Customers' group, which are customers, who have returned after their first order with vouchers within a year.

Bad Voucher customers: 
- First_Order_Voucher == TRUE
- Completed_Orders == 1
- Total_Orders == 1
- Last_Order > 365 days ago

Good Voucher customers:
- First_Order_Voucher == TRUE
- Completed_Orders > 1
- Second completed order within a year

In [ ]:
df_vou = df.loc[df.first_order_voucher & df.completed_orders > 0]
df_vou_good = df_vou.loc[(df_vou.completed_orders > 1) & (df_vou.first_order_recency < 365)]
df_vou_bad = df_vou.loc[(df_vou.completed_orders == 1) & (df_vou.total_orders == 1) & (df_vou.recency > 365)]

df_vou_bad = df_vou_bad.loc[df_vou_bad.first_order_date < "2017-08-29"]
df_vou_good = df_vou_good.loc[df_vou_good.first_order_date < "2017-08-29"]

In [ ]:
print('Number of Bad customers: ', df_vou_bad.shape[0])
print('Number of Good customers: ', df_vou_good.shape[0])

df_vou_bad['customer_type'] = 'bad'
df_vou_good['customer_type'] = 'good'

In [ ]:
df_vou.refunds_success.value_counts()

# Bad customers with Bad experience
*Assumption: If a BAD customer churned because of bad experience, he should be included in the churn analysis*

Customers falling into the following groups should not be excluded from analysis:
* reclean_order
* order_rating < 4
* internal_reschedules > 1
* refund
* order was unpunctual

In [ ]:
df_vou_bad.groupby(['reclean_orders'])['customer_db_id'].nunique()

In [ ]:
df_vou_bad.groupby(['last_order_rating'])['customer_db_id'].nunique()

In [ ]:
df_vou_bad.groupby(['internal_reschedules'])['customer_db_id'].nunique()

In [ ]:
df_vou_bad.groupby(['refunds_success'])['customer_db_id'].nunique()

In [ ]:
df_vou_bad.groupby(['unpunctual_orders'])['customer_db_id'].nunique()

In [ ]:
df_both = pd.concat([df_vou_bad, df_vou_good])
df_both['customer_type_enc'] = df_both['customer_type'].map({'bad': -1, 'good': 1})

df_both['experience'] = 'good'
df_both.loc[
    (df_both.reclean_orders == 1) |
    ((df_both.last_order_rating < 4) & (df_both.last_order_rating > 0)) |
    (df_both.refunds_success == 1) |
    (df_both.internal_reschedules > 1) |
    (df_both.unpunctual_orders == 1), 'experience'] = 'bad'

print('Bad with bad experience: ', df_both.loc[(df_both.customer_type == 'bad') & (df_both.experience == 'bad')].shape[0])
print('Bad with good experience: ',df_both.loc[(df_both.customer_type == 'bad') & (df_both.experience == 'good')].shape[0])

### Bad experience Facilities

In [ ]:
(df_both.loc[df_both.experience == 'bad', 'last_order_fac_name'].value_counts() 
 / df_both.last_order_fac_name.value_counts()).sort_values()

In [ ]:
df_both.to_csv('./data/pbi/bad_data.csv', index=False)

# Bad Customers with Good Experience

In [ ]:
def plot_bad_vs_good_by_column(df_both, column_name, print_df=False):
    df_bad = df_both.loc[df_both.customer_type == 'bad']\
                .groupby(column_name)['customer_db_id']\
                .nunique() / df_vou_bad.shape[0]
    df_good = df_both.loc[df_both.customer_type == 'good']\
                .groupby(column_name)['customer_db_id']\
                .nunique() / df_vou_good.shape[0]
    
    df_diff = df_good - df_bad
    df_diff_relative = df_diff / (df_bad + df_good)
    df = pd.concat([df_bad, df_good, df_diff, df_diff_relative], 
                  axis=1, keys=['bad', 'good', 'diff', 'diff_relative'], sort=True)
    
    if print_df:
        print(df)
    
    df = df.sort_values('diff', ascending=False)
    df[['bad', 'good']].plot(kind='bar', title=column_name)
    df[['diff']].plot(kind='bar', title='diff (+ more good / - more bad)', color='orange')
#     df[['diff_relative']].sort_values('diff_relative', ascending=False).plot(kind='bar', title='diff relative (diff / all customers)', color='orange')

In [ ]:
def plot_bad_vs_good_dist(df_both, column_name):
    sns.categorical.violinplot(data=df_both, x='customer_type', y=column_name)

    f, (ax1, ax2) = plt.subplots(2, sharex=True, sharey=True,figsize=(6,6))
    sns.distplot(df_both.loc[df_both.customer_type == 'bad', column_name], ax=ax1, color='red', kde=False, bins=10)
    ax1.set_title('Bad Customers')
    sns.distplot(df_both.loc[df_both.customer_type == 'good', column_name], ax=ax2, color='blue', kde=False, bins=10)
    ax2.set_title('Good Customers')
    plt.tight_layout()

## Cluster

In [ ]:
plot_bad_vs_good_dist(df_both.loc[(df_both.cluster != -999)], 'cluster')

In [ ]:
plot_bad_vs_good_by_column(df_both.loc[(df_both.cluster != -999)], 'cluster_name')

## Voucher Channel
*Assumption: The channel through which the customer has received the voucher influences if the acquired customer is going to come back after the first order.*

In [ ]:
plot_bad_vs_good_by_column(df_both.loc[df_both.experience == 'good'], 'first_order_voucher_channel', False)

In [ ]:
df_both.groupby(['customer_type', 'experience', 'first_order_voucher_channel'])\
        ['customer_db_id'].nunique().reset_index()\
        .to_csv('./data/pbi/bad_cust.csv', index=False)

In [ ]:
df_both.loc[(df_both.customer_type == 'bad') & (df_both.experience == 'good')].first_order_voucher_channel.value_counts()

Conclusion: Customers that have acquired their vouchers from the following channels are **likely to churn after first order, even if they had a good customer experience**, and should be excluded from the analysis.
* SEO
* SEM
* Appco/Direct Sales



In [ ]:
bad_voucher_channels = ['SEO', 'SEM', 'Appco/Direct Sales']

## Voucher Value

In [ ]:
plot_bad_vs_good_dist(df_both, 'first_order_voucher_revenue_ratio')

In [ ]:
df_both.loc[df_both.first_order_voucher_revenue_ratio > 0.9, 'customer_type'].value_counts()

In [ ]:
df_both.loc[(df_both.first_order_voucher_channel == 'Appco/Direct Sales') & 
            (df_both.first_order_voucher_revenue_ratio > 0.9), 
            'customer_type'].value_counts()

In [ ]:
plot_bad_vs_good_by_column(df_both.loc[df_both.first_order_voucher_revenue_ratio > 0.9], 'first_order_voucher_channel')

## First Order Revenue
*Assumption: Depending on how much the customer actually spent on their first voucher order, determines if they are going to order again or not. Customer who spend more, tend to be good customers, because they can afford our service regularly*

In [ ]:
a = df_both[['customer_type', 'first_order_revenue']]
sns.categorical.violinplot(data=a, x='customer_type', y='first_order_revenue')

In [ ]:
a = df_both[['customer_type', 'first_order_voucher_revenue_ratio']]
sns.categorical.violinplot(data=a, x='customer_type', y='first_order_voucher_revenue_ratio')

f, (ax1, ax2) = plt.subplots(2, sharex=True, sharey=True,figsize=(6,6))
sns.distplot(a.loc[a.customer_type == 'bad', 'first_order_voucher_revenue_ratio'], ax=ax1, color='red', bins=50)
ax1.set_title('Bad Customers')
sns.distplot(a.loc[a.customer_type == 'good', 'first_order_voucher_revenue_ratio'], ax=ax2, color='blue', bins=50)
ax2.set_title('Good Customers')
plt.tight_layout()

a.groupby('customer_type').describe().T

In [ ]:
sns.distplot(df_both.loc[df_both.customer_type=='bad'].laundry_distance)

## Distance to Laundry

In [ ]:
a = df_both.loc[(df_both.laundry_within_1km != -999) & (df_both.experience == 'good'), ['customer_type', 'laundry_distance']]
# sns.categorical.violinplot(data=a, x='customer_type', y='laundry_distance')

f, (ax1, ax2) = plt.subplots(2, sharex=True, sharey=True,figsize=(6,6))
sns.distplot(a.loc[a.customer_type == 'bad', 'laundry_distance'], ax=ax1, color='red', bins=7)
ax1.set_title('Bad Customers')
sns.distplot(a.loc[a.customer_type == 'good', 'laundry_distance'], ax=ax2, color='blue', bins=7)
ax2.set_title('Good Customers')
plt.tight_layout()

In [ ]:
plot_bad_vs_good_by_column(df_both.loc[df_both.city == "London"], 'zip')

*Conclusion*: It doesn't seem that distance to hub plays a role in the first_voucher_order churn.

In [ ]:
a = df_both.loc[df_both.first_order_hub_distance != -999, ['customer_type', 'first_order_hub_distance']]
sns.categorical.violinplot(data=a, x='customer_type', y='first_order_hub_distance')

f, (ax1, ax2) = plt.subplots(2, sharex=True, sharey=True,figsize=(6,6))
sns.distplot(a.loc[a.customer_type == 'bad', 'first_order_hub_distance'], ax=ax1, color='red', bins=25)
ax1.set_title('Bad Customers')
sns.distplot(a.loc[a.customer_type == 'good', 'first_order_hub_distance'], ax=ax2, color='blue', bins=25)
ax2.set_title('Good Customers')
plt.tight_layout()

## Dates
*Assumption: The dates when more bad customers came than good, could mean that there was a certain period of campaigns that attracted undesirable customers.*

In [ ]:
bad_limit = df_vou_bad.loc[df_vou_bad.first_order_date < "2017-07-21"]
good_limit = df_vou_good.loc[df_vou_good.first_order_date < "2017-07-21"]
bad = bad_limit\
        .groupby('first_order_date')['customer_db_id']\
        .nunique() / bad_limit.shape[0]
good = good_limit\
        .groupby('first_order_date')['customer_db_id']\
        .nunique() / good_limit.shape[0]

df_plot = pd.concat([bad, good], axis=1, keys=['bad', 'good']).reset_index()
df_plot = df_plot.groupby([pd.Grouper(key='first_order_date', freq='MS')])[['bad', 'good']]\
        .sum()\
        .reset_index()\
        .sort_values('first_order_date')\
        .set_index('first_order_date')

In [ ]:
df_plot.plot()

*Conclusion*: Seems that there were some areas, especially around Feb - April 2016, where loads of "bad customers" were coming in. We should look at what kind of campaigns there were running at that time.

## Facility
*Assumption: Some facilities provide better services and therefore customers who churned after first order, although having a 'good' facility, are concidered bad.*

In [ ]:
df_bad = df_vou_bad.loc[df_vou_bad.first_order_fac_name != "unknown"]\
        .groupby('first_order_fac_name')['customer_db_id'].nunique() / df_vou_bad.shape[0]
df_good = df_vou_good.loc[df_vou_good.first_order_fac_name != "unknown"]\
        .groupby('first_order_fac_name')['customer_db_id'].nunique() / df_vou_good.shape[0]
df_bad_exp = df_vou_bad_exp.loc[df_vou_bad_exp.first_order_fac_name != "unknown"]\
        .groupby('first_order_fac_name')['customer_db_id'].nunique() / df_vou_bad_exp.shape[0]
df_diff = df_good - df_bad
df_diff_relative = df_diff / (df_bad + df_good)
df = pd.concat([df_bad, df_good, df_bad_exp, df_diff, df_diff_relative], 
              axis=1, keys=['bad', 'good', 'bad_exp', 'diff', 'diff_relative'], sort=True)

df = df.sort_values('diff', ascending=False)
df[['bad', 'good', 'bad_exp']].plot(kind='bar', title='first_order_fac_name')
df[['diff']].plot(kind='bar', title='diff (+ more good / - more bad)', color='orange')

In [ ]:
df_bad = df_vou_bad.loc[df_vou_bad.first_order_fac_name != "unknown"]\
        .groupby('first_order_fac_name')['customer_db_id'].nunique() / df_vou_bad.shape[0]

## Product Segmentation
*Assumption: Bad customers order certain type of products*

TODO: too many UNKNOWN values for bad customers, need to include itemization

In [ ]:
plot_bad_vs_good_by_column(df_both, 'first_order_products')

In [ ]:
plot_bad_vs_good_by_column(df_both.loc[df_both.cluster != -999], 'cluster_name')

In [ ]:
df_both

## Dates
*Assumption: There is a certain pattern in the recency of the second order for the good customers.*

In [ ]:
sns.distplot(df_vou_good.first_order_recency)

In [ ]:
df_vou_good.first_order_recency.describe()

# Distance to Laundry

In [ ]:
a = df_both.loc[(df_both.laundry_distance != -999), ['customer_type', 'laundry_distance']]
sns.categorical.violinplot(data=a, x='customer_type', y='laundry_distance')

f, (ax1, ax2) = plt.subplots(2, sharex=True, sharey=True,figsize=(6,6))
sns.distplot(a.loc[a.customer_type == 'bad', 'laundry_distance'], ax=ax1, color='red', bins=25)
ax1.set_title('Bad Customers')
sns.distplot(a.loc[a.customer_type == 'good', 'laundry_distance'], ax=ax2, color='blue', bins=25)
ax2.set_title('Good Customers')
plt.tight_layout()